In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
raw_loc_base = '../data/raw'

In [3]:
master_name = 'Input information and X10 - X18.xlsx'
master_loc = os.path.join(raw_loc_base, master_name)
master = pd.read_excel(master_loc, sheet='inputs', header=0)

In [4]:
master.head()

,Input,Key,X10: Category Method,X11: Temperature (K),X12: [Salt*Valency],X13: Category Salt type,X14: [Buffer] (mM),X15: pH,X16: CI number,X17: CI,X18: CP,Output: logK
0,1,17,A,298,0,0,0,6.00,0,NaN,CP_3,4.39
1,2,17,B,298,0,0,50,4.74,0,NaN,NaN,4.39
2,3,19,A,298,0,0,0,6.00,1,CI_2,CP_3,4.63
3,4,20,A,298,0,0,0,6.00,2,CI_1,CP_3,4.80
4,5,21,A,298,0,0,0,6.00,2,CI_2,CP_3,4.90


In [5]:
def read_X1(file_name, sheet_name='X1', col_names=['atom_number', 'AMU', 'x', 'y', 'z']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)
    
def read_X2(file_name, sheet_name='X2', col_name='occupied'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)

def read_X2V(file_name, sheet_name='X2V', col_name='virtual'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)

def read_X3(file_name, sheet_name='X3'):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None)

def read_X4(file_name, sheet_name='X4'):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None)

def read_X5(file_name, sheet_name='X5', col_names=['electric_potential', 'x', 'y', 'z']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)

def read_X6(file_name, sheet_name='X6', col_names=['XX', 'YY', 'ZZ', 'XY', 'XZ', 'YZ']):
    return pd.read_excel(file_name, sheet_name=sheet_name, header=None, names=col_names)

def read_X7(file_name, sheet_name='X7'):
    return pd.read_excel(file_name, sheet_name='X7', header=None)

def read_X8(file_name, sheet_name='X8', col_name='SCF'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)
    
def read_X9(file_name, sheet_name='X9', col_name='MO'):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    flat = df.values.flatten()
    return pd.Series(flat[~np.isnan(flat)]).rename(col_name)

In [7]:
x = read_X2V(os.path.join(raw_loc_base, '19.xlsx'))
x

0      -0.19713
1      -0.14132
2      -0.13880
3      -0.13156
4      -0.12411
5      -0.11555
6      -0.09972
7      -0.09322
8      -0.09085
9      -0.08373
10     -0.07465
11     -0.07136
12     -0.06366
13     -0.05436
14     -0.04371
15     -0.04098
16     -0.03595
17     -0.03264
18     -0.02805
19     -0.01696
20     -0.01316
21     -0.00529
22     -0.00356
23     -0.00126
24      0.00335
25      0.00643
26      0.01194
27      0.02103
28      0.02485
29      0.02556
         ...   
254     3.68720
255     3.82042
256     3.82945
257     3.93519
258     4.00320
259     4.03655
260     4.05836
261     4.08363
262     4.19633
263     4.42964
264     4.43661
265     4.49620
266     4.74000
267     4.86510
268     4.93895
269     5.30931
270     5.37457
271     5.61745
272     5.72462
273    23.45961
274    23.65547
275    23.74926
276    23.81266
277    23.82542
278    23.94418
279    23.98364
280    24.06722
281    35.31211
282    49.79912
283    49.87500
Name: virtual, Length: 2